In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.impute import SimpleImputer 
from sklearn.tree import DecisionTreeClassifier

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [98]:
df=pd.read_csv("titanic.csv")

In [102]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [104]:
df=df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
type(df)

pandas.core.frame.DataFrame

In [80]:
#PLAN (IMPUTE MISSING VALUES > ONE HOT ENCODING > SCALING > FEATURE SELECTION > MODEL TRAIN USING DECISION TREE

In [108]:
x_train, x_test, y_train, y_test= train_test_split(df.drop(columns=["Survived"]), df["Survived"], test_size=0.2, random_state=42)

In [110]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [116]:
#IMPUTATION TRANSFORMER 
trf1=ColumnTransformer([("imputed_age", SimpleImputer(), [2]), 
                       ("imputed_embarked", SimpleImputer(strategy="most_frequent"), [6])], remainder="passthrough")

In [118]:
#one hot encoding 
trf2=ColumnTransformer([("ohe_sex_embarked", OneHotEncoder(sparse=False, handle_unknown="ignore"), [1,6])]
                       , remainder="passthrough")

In [128]:
#Scaling
trf3=ColumnTransformer([("scaled", MinMaxScaler(), slice(0,10))])

In [126]:
#feature selection 
trf4=SelectKBest(score_func=chi2, k=10)

In [130]:
#train the model
trf5=DecisionTreeClassifier()

CREATE PIPELINE

In [193]:
pipe=Pipeline([("trf1", trf1), ("trf2", trf2), ("trf3", trf3), ("trf4", trf4), ("trf5", trf5)])

<h1>PIPELINE VS make_pipeline<h1><h6>Pipeline requires naming of steps, make_pipeline does not (same is with ColumnTransformer vs make_column_transformer)<h6>

In [195]:
#train 
pipe.fit(x_train, y_train)

C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_age',
                                                  SimpleImputer(), [2]),
                                                 ('imputed_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scaled', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(score_func=<function chi2 at 0x00000192BB601F80>)),
                ('trf5', DecisionTreeClassifier())])

In [179]:
from sklearn import set_config
set_config(display="diagram")

In [197]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('imputed_age', SimpleImputer(), [2]),
                                 ('imputed_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scaled', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(score_func=<function chi2 at 0x00000192BB601F80>),
 'trf5': DecisionTreeClassifier()}

In [203]:
pipe.named_steps["trf3"]

ColumnTransformer(transformers=[('scaled', MinMaxScaler(), slice(0, 10, None))])

In [165]:
y_predict=pipe.predict(x_test)

In [221]:
pipe.named_steps["trf1"].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [223]:
pipe.named_steps["trf1"].transformers_[0][1].statistics_ 

array([29.49884615])

In [227]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict) #less accuracy because we removed rows using feature selection

0.6256983240223464

<h1>Cross Validation using Pipeline<h1>

In [234]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring="accuracy").mean()

C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1

0.6391214419383433

In [236]:
import pickle 
import numpy as np


In [240]:
#pipe=pickle.load(open("pipe.pkl", "rb"))

In [252]:
#assume user input
test_input=np.array([2, "male", 31.0,0,0,10.5, "S"], dtype=object).reshape(1,7)

In [254]:
pipe.predict(test_input)

C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\NitroV\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)